In [1]:
import numpy as np
import os
import pandas as pd
import rasterio
from glob import glob
from tqdm import tqdm

In [2]:
workspace = r"D:\物候分析\气象湿度\栅格数据"
outspace = r"D:\物候分析\气象湿度"
items = ['pre*.tif', 'soil_tem*.tif']

In [3]:
def func(path):
    with rasterio.open(path) as src:
        profile = src.profile
        profile['dtype'] = 'float32'
        nodata = profile['nodata']
        dt = np.float32(src.read(1))
        dt[dt==nodata] = np.nan
        return dt

In [4]:
with rasterio.open(r"D:\物候分析\气象湿度\栅格数据\pre_2001.tif") as mode:
    mode_profile = mode.profile
    mode_profile['dtype'] = 'float32'

In [6]:
for item in tqdm(items, desc='物候类型'):
    rasters = glob(os.path.join(workspace, item))
    arr_sum = np.zeros((mode_profile['height'], mode_profile['width']))
    count = np.zeros((mode_profile['height'], mode_profile['width']))
    
    for i, raster in enumerate(tqdm(rasters, desc='栅格求和')):
        arr = func(raster)
        arr_sum = np.where(~np.isnan(arr), arr_sum+arr, arr_sum)
        count = count + ~np.isnan(arr)
        
    result = np.where(count!=0, arr_sum/count, np.nan)
    name = 'avg_' + item.replace('out_', '').replace('*', '')
    with rasterio.open(os.path.join(outspace, name), 'w', **mode_profile) as outfile:
        outfile.write(result, 1)

物候类型：:   0%|          | 0/2 [00:00<?, ?it/s]

栅格求和：: 100%|██████████| 24/24 [00:03<00:00,  6.17it/s]
C:\Users\Qiu\AppData\Local\Temp\ipykernel_39328\2326610049.py:11: RuntimeWarning: invalid value encountered in divide
  result = np.where(count!=0, arr_sum/count, np.nan)
物候类型：: 100%|██████████| 2/2 [00:08<00:00,  4.26s/it]


In [9]:
sog = glob(r"D:\物候分析\物候提取\提取后\out_sos*.tif")
eog = glob(r"D:\物候分析\物候提取\提取后\out_eos*.tif")
dt = pd.DataFrame()
lst = []
for i in tqdm(range(len(sog))):
    sog_raster = func(sog[i])
    eog_raster = func(eog[i])
    divide = eog_raster-sog_raster
    divide = np.where(divide<=0, np.nan, divide)
    lst.append(np.nanmean(divide))
dt['log'] = lst
dt.index = [i for i in range(2001, 2025)]
dt.to_csv(r"D:\物候分析\物候提取\生长季长度均值.csv", index=False)

100%|██████████| 24/24 [00:56<00:00,  2.36s/it]
